In [ ]:
!pip install transformers[sentencepiece]

     |████████████████████████████████| 4.0 MB 8.1 MB/s 
     |████████████████████████████████| 895 kB 30.4 MB/s 
     |████████████████████████████████| 6.6 MB 10.2 MB/s 
     |████████████████████████████████| 77 kB 5.8 MB/s 
     |████████████████████████████████| 596 kB 46.4 MB/s 
     |████████████████████████████████| 1.2 MB 51.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install datasets

     |████████████████████████████████| 325 kB 7.2 MB/s 
     |████████████████████████████████| 136 kB 67.4 MB/s 
     |████████████████████████████████| 1.1 MB 68.7 MB/s 
     |████████████████████████████████| 212 kB 55.0 MB/s 
     |████████████████████████████████| 127 kB 48.3 MB/s 
     |████████████████████████████████| 271 kB 62.1 MB/s 
     |████████████████████████████████| 144 kB 45.7 MB/s 
     |████████████████████████████████| 94 kB 1.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [ ]:
import numpy as np
import pandas as pd

# DS_DIR = './'
DS_DIR = '/content/gdrive/My Drive/'
label = 'text1'
models = ['google', 'wit', 'deepgram']


def load_data(model):
    df = pd.read_csv(DS_DIR + 'ljs-data.csv')
    train_df, valid_df = np.split(df.sample(frac=1, random_state=42), [int(.8 * len(df))])
    errors_train = train_df['google'].isna().values | train_df['wit'].isna().values | train_df['deepgram'].isna().values
    y_train = train_df[label].values
    X_train = train_df[model].values
    errors_valid = valid_df['google'].isna().values | valid_df['wit'].isna().values | valid_df['deepgram'].isna().values
    y_valid = valid_df[label].values
    X_valid = valid_df[model].values
    return X_train[~errors_train], y_train[~errors_train], X_valid[~errors_valid], y_valid[~errors_valid]

In [ ]:
X1_train, y1_train, X1_valid, y1_valid = load_data('google')
X2_train, y2_train, X2_valid, y2_valid = load_data('wit')

In [ ]:
from datasets import Dataset

df_train = pd.DataFrame({"text1": X1_train, "text2": X2_train, "labels": y1_train})
dataset_train = Dataset.from_pandas(df_train)

df_valid = pd.DataFrame({"text1": X1_valid, "text2": X2_valid, "labels": y1_valid})
dataset_valid = Dataset.from_pandas(df_valid)

In [ ]:
prefix = "merge sentence1: "
infix = " sentence2: "

def preprocess_function(examples):
    inputs = [prefix + examples["text1"][i] + infix + examples["text2"][i] for i in range(len(examples["text1"]))]
    model_inputs = tokenizer(inputs, padding="max_length", max_length=320, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['labels'], padding="max_length", max_length=320, truncation=True)

    # model_inputs["input_ids"] = model_inputs["input_ids"].squeeze(0)
    # model_inputs["attention_mask"] = model_inputs["attention_mask"].squeeze(0)
    model_inputs["labels"] = labels["input_ids"] #.squeeze(0)

    return model_inputs

tokenized_datasets_train = dataset_train.map(preprocess_function, batched=True)
tokenized_datasets_valid = dataset_valid.map(preprocess_function, batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets_train = tokenized_datasets_train.remove_columns(["text1", "text2"])
tokenized_datasets_valid = tokenized_datasets_valid.remove_columns(["text1", "text2"])
# tokenized_datasets.set_format("torch")
tokenized_datasets_train

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 10477
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results-google-wit",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=100,
    fp16=True,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_valid,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Using amp half precision backend


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10477
  Num Epochs = 100
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 65500


Epoch,Training Loss,Validation Loss
1,0.597500,0.055658
2,0.067300,0.046403
3,0.055200,0.043202
4,0.048500,0.041163
5,0.046400,0.039217
6,0.044700,0.037905
7,0.041800,0.036930
8,0.040700,0.036020
9,0.039600,0.035286
10,0.037700,0.034691


Saving model checkpoint to ./results-google-wit/checkpoint-500
Configuration saved in ./results-google-wit/checkpoint-500/config.json
Model weights saved in ./results-google-wit/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results-google-wit/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results-google-wit/checkpoint-500/special_tokens_map.json
Copy vocab file to ./results-google-wit/checkpoint-500/spiece.model
***** Running Evaluation *****
  Num examples = 2615
  Batch size = 16
Saving model checkpoint to ./results-google-wit/checkpoint-1000
Configuration saved in ./results-google-wit/checkpoint-1000/config.json
Model weights saved in ./results-google-wit/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results-google-wit/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results-google-wit/checkpoint-1000/special_tokens_map.json
Copy vocab file to ./results-google-wit/checkpoint-1000/spiece.model
***** Run

TrainOutput(global_step=65500, training_loss=0.02944617520397856, metrics={'train_runtime': 24597.8255, 'train_samples_per_second': 42.593, 'train_steps_per_second': 2.663, 'total_flos': 8.8623503376384e+16, 'train_loss': 0.02944617520397856, 'epoch': 100.0})

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

results = []

for batch in tokenized_datasets_valid:
  y_pred = model.generate(torch.tensor([batch['input_ids']]).to(device))

  for i in range(len(y_pred)):
    results.append({
      'input': tokenizer.decode(batch['input_ids'], skip_special_tokens=True),
      'output': tokenizer.decode(y_pred[i], skip_special_tokens=True),
      'label': tokenizer.decode(batch['labels'], skip_special_tokens=True),
    })

results_df = pd.DataFrame(results)
results_df.to_csv(DS_DIR + 'google-wit-results.csv')
